### Clean CSV

In [ ]:
import pandas as pd

stops_df = pd.read_csv("./data/stops.csv")
stops_df = stops_df[["stop_id", "stop_name", "stop_lat", "stop_lon"]]
stops_df = stops_df.rename(columns=lambda x: x.replace("stop_", ""))
stops_df = stops_df.rename(columns={"name": "address"})

stops_df["route"] = stops_df["address"].apply(lambda x: x.split(",")[0])
stops_df["street_number"] = stops_df["address"].apply(lambda x: x.split(",")[-1])

stops_df.to_csv("./data/stops.tgh.csv", index=False)

## Street View

### Import and Helpers

In [ ]:
import pandas as pd

from os import makedirs, path
from time import sleep

from utils import get_address, get_latlon
from utils import get_bus_station, get_place_latlon
from utils import get_heading, get_streetview_image

from env import GOOGLE_API_KEY

for dir in ["address", "address_coords", "heading", "station_coords"]:
  makedirs(f"./imgs/{dir}", exist_ok=True)

### Test coords vs address

In [ ]:
stops_df = pd.read_csv("./data/stops.tgh.csv")
stops_df

In [ ]:
for idx,row in stops_df.iterrows():
  if idx > 1023:
    break

  fname = str(row["id"]) + ".jpg"
  address_path = path.join("./imgs/address", fname)
  address_coords_path = path.join("./imgs/address_coords", fname)
  heading_path = path.join("./imgs/heading", fname)
  station_coords_path = path.join("./imgs/station_coords", fname)

  if not path.isfile(address_path):
    address_img = get_streetview_image(GOOGLE_API_KEY, address=row["address"])
    address_img.save(address_path)
    sleep(0.1)

  if not path.isfile(address_coords_path):
    coords_img = get_streetview_image(GOOGLE_API_KEY, lat=row["lat"], lon=row["lon"])
    coords_img.save(address_coords_path)
    sleep(0.1)

  if path.isfile(heading_path) and path.isfile(station_coords_path):
    continue

  response = get_bus_station(GOOGLE_API_KEY, lat=row["lat"], lon=row["lon"], radius=30)
  if "places" in response:
    station = response["places"][0]
    station_latlon = get_place_latlon(station)

    if not path.isfile(heading_path):
      heading = get_heading((row["lat"], row["lon"]), station_latlon)
      coords_img = get_streetview_image(GOOGLE_API_KEY, lat=row["lat"], lon=row["lon"], heading=heading)
      coords_img.save(heading_path)
      sleep(0.1)

    if not path.isfile(station_coords_path):
      scoord_img = get_streetview_image(GOOGLE_API_KEY, lat=station_latlon[0], lon=station_latlon[1])
      scoord_img.save(station_coords_path)
      sleep(0.1)

In [ ]:
row = stops_df.iloc[0]
display(row)
display(get_address(GOOGLE_API_KEY, lat=row["lat"], lon=row["lon"]))
display(get_latlon(GOOGLE_API_KEY, address=f"{row['address']}, Fortaleza, Ceara, Brazil"))
display(get_bus_station(GOOGLE_API_KEY, lat=row["lat"], lon=row["lon"], radius=8))

In [ ]:
places = get_bus_station(GOOGLE_API_KEY, lat=row["lat"], lon=row["lon"], radius=8)["places"]
bus_station_latlon = get_place_latlon(places[0])
get_heading((row["lat"], row["lon"]), bus_station_latlon)